# Feature extraction with the Candidate Tower

After Two-Tower training, the `candidate_tower` is used to convert all candidate items into embeddings. 

The embeddings are indexed and deployed to an endpoint for serving.

In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 

In [2]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_io as tfio

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

## Load `SavedModel`

In [3]:
# gcs uri for SavedModel dir
# candidate_tower_uri = 'gs://spotify-tfrs-dir/v2/run-20220920-210334/candidate_tower' # locally trained
candidate_tower_uri = 'gs://spotify-tfrs-dir/v11/run-20220921-163503/candidate_tower' # vertex trained

loaded_candidate_model = tf.saved_model.load(candidate_tower_uri)

loaded_candidate_model.signatures

2022-09-21 17:15:40.618131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-09-21 17:15:40.618285: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-21 17:15:40.618332: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jt-tfrs-spotify-sept-v2): /proc/driver/nvidia/version does not exist
2022-09-21 17:15:40.619064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, album_name_can, album_uri_can, artist_followers_can, artist_genres_can, artist_name_can, artist_pop_can, artist_uri_can, duration_ms_can, track_name_can, track_pop_can, track_uri_can) at 0x7FB8AB7C8FD0>})

In [ ]:
print(list(loaded_candidate_model.signatures.keys()))

In [ ]:
candidate_predictor = loaded_candidate_model.signatures["serving_default"]
print(candidate_predictor.structured_outputs)

In [ ]:
candidate_predictor.output_shapes

In [ ]:
candidate_features = {
        'track_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_name_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'track_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'album_uri_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'duration_ms_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'track_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_pop_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    'artist_genres_can': tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    'artist_followers_can': tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
}